In [12]:
from urllib.request import urlopen
import urllib.request as req
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

from pandas import Series, DataFrame
import pandas as pd
import numpy as np
import datetime

In [ ]:
# 자동화코드
# 첫번째 if문: li의 개수가 10개임에 불구하고 수집과정에는 10개가 되지않아 에러가 발생
# 두번째 if문: 네이버 리뷰의 경우 리뷰의 내용이 길면 더보기를 눌러야만 전체 내용이 수집됨,
#              따라서 더뷰기가 있는 경우, 'span._unfold_ment > a'가 존재하기에 이것을 수집

score = []
review = []
id = []
date = []

for page in range(1, 805):
    url = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=190010&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page='+str(page)
    html = urlopen(url)
    soup = BeautifulSoup(html, "html.parser")
    
    li = soup.select('div.ifr_area.basic_ifr > div.input_netizen  > div.score_result > ul > li')

    if len(li) != 10:
        page = page + 1
    else:
        for i in li:
            score.append(int(i.select('div.star_score > em')[0].text))
            id.append(i.select('div.score_reple > dl > dt')[0].text.strip()[:-19])
            date.append(i.select('div.score_reple > dl > dt')[0].text.strip()[-16:-6])
        for i in range(0,10):
            span = li[i].select('div.score_reple > p > span#_filtered_ment_'+str(i))
            if span[0].select('span._unfold_ment > a') == []:
                review.append(li[i].select('div.score_reple > p')[0].text.strip().lstrip('관람객').strip().lstrip('스포일러가 포함된 감상평입니다. 감상평 보기').strip())
            else:
                review.append(span[0].select('span._unfold_ment > a')[0]['data-src'])
        page = page + 1

movie = DataFrame({'date': date, 'id': id, 'review': review, 'score': score})

In [ ]:
# review 값이 없는 행 제거  -> 7455개
movie = movie.drop(movie[movie.review == ''].index)

In [9]:
import pickle

In [ ]:
# 변수형태 그대로 바이너리 파일로 저장하기
with open('c:/data/movie_tenet.pkl', 'wb') as file:
    pickle.dump(movie, file)

In [9]:
# 바이너리파일 불러오기
with open('c:/data/movie_tenet.pkl', 'rb') as file:
    movie_tenet = pickle.load(file)

movie = movie_tenet

In [ ]:
# -------------------점수별 리뷰 수-------------------
df1 = movie.review.groupby(movie.score).count()

df1 = df1.reset_index()
df1.columns = ['score', 'count']

In [ ]:
import matplotlib.pylab as plt
from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname='c:\windows/fonts/malgun.ttf').get_name()
rc('font', family=font_name)

plt.bar(df1['score'], df1['count'], color = 'green')
plt.xticks(df1.score)
plt.title('점수별 리뷰 수 현황', size=10)
plt.xlabel('점수', size=8)
plt.ylabel('리뷰수', size=8)
for i in df1.index:
    plt.text(i+0.75, df1['count'][i]+7, df1['count'][i])

np.mean(movie.score)

In [ ]:
# -------------------요일별 리뷰 수-------------------

week = []
for i in movie.date:
    week.append(int(datetime.datetime.strptime(i, '%Y.%m.%d').strftime('%w')))
movie['week'] = week
movie.review.groupby(movie.week).count()

In [ ]:
df2 = movie.review.groupby(movie.week).count()

df2 = df2.reset_index()
df2.columns = ['weekday', 'count']

In [ ]:
import matplotlib.pylab as plt
from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname='c:\windows/fonts/malgun.ttf').get_name()
rc('font', family=font_name)

plt.bar(df2['weekday'], df2['count'], color = '#de956d')
plt.xticks(df2.weekday, '일월화수목금토')
plt.title('요일별 리뷰 수', size=10)
#lt.xlabel('요일', size=8)
plt.ylabel('리뷰수', size=8)
for i in df2.index:
    plt.text(i-0.2, df2['count'][i]+5, df2['count'][i])

In [ ]:
# -------------------날짜별 리뷰 수-------------------
movie.review.groupby(movie.date).count()

In [ ]:
import datetime

days = []
for i in movie['date']:
    days.append((datetime.datetime.strptime(i, '%Y.%m.%d') - datetime.datetime(2020,8,26)).days)
movie['days'] = days

In [ ]:
df3 = movie.review.groupby(movie.days).count()
plt.plot(df3.index, df3, color='#de956d', linestyle='-')
plt.title('날짜별 리뷰 수', size=10)
plt.ylabel('리뷰수', size=8)
plt.vlines(50, -3, 910, color = 'black')

In [ ]:
# 50일 이전만 확대 그래프
df3_50 = df3.sort_index().head(n=50)

plt.plot(df3_50.index, df3_50, color='#de956d', linestyle='-')
plt.title('개봉 50일 이내의 날짜별 리뷰 수', size=10)
plt.ylabel('리뷰수', size=8)
plt.text(0.3, df3_50[0]-15, df3_50[0])
plt.text(9.8, df3_50[11]+10, df3_50[11])